<a href="https://colab.research.google.com/github/davhofer/recommender/blob/main/ncf_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.7 MB/s eta 0:00:00


In [4]:
# Import the tables of the data set as dataframes.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

DATA_DIR = '/content/drive/MyDrive/data' 
STUDY_DIR = DATA_DIR + '/study'


users = pd.read_csv(f'{DATA_DIR}/users.csv.gz')

# use study for less data, for testing
events = pd.read_csv(f'{DATA_DIR}/events.csv.gz')
transactions = pd.read_csv(f'{DATA_DIR}/transactions.csv.gz')

In [6]:
from data import preprocess_transactions, get_transactions_dataloader

train_transactions, test_transactions, user_ids, topic_ids = preprocess_transactions(transactions)

train_dataloader = get_transactions_dataloader(train_transactions, user_ids, topic_ids, 128)

test_dataloader = get_transactions_dataloader(test_transactions, user_ids, topic_ids, 128)

In [19]:
import torch
from torch import optim, nn
import lightning.pytorch as pl
from ncf_model import NCFNetwork

num_students = len(user_ids)
num_topics = len(topic_ids)

student_embedding_dim = 32
topic_embedding_dim = 32

predictive_factors = 32

ncf = NCFNetwork(num_students, num_topics, student_embedding_dim, topic_embedding_dim, predictive_factors)


In [20]:
trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        accumulate_grad_batches=10,
        max_epochs=1,
)

trainer.fit(model=ncf, train_dataloaders=train_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type       | Params
-------------------------------------------------------
0 | student_embedding_layer | Linear     | 465 K 
1 | topic_embedding_layer   | Linear     | 2.4 K 
2 | network                 | Sequential | 2.8 K 
3 | loss                    | MSELoss    | 0     
-------------------------------------------------------
471 K     Trainable params
0         Non-trainable params
471 K     Total params
1.884     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [21]:
trainer.test(model=ncf, dataloaders=test_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{}]

In [22]:
ncf.predict_proba

tensor([[0.5345],
        [0.5316],
        [0.5117],
        ...,
        [0.5134],
        [0.5352],
        [0.5141]])